In [60]:
import sys
sys.path.append('/dfs/scratch1/simran/tutorial/contextual-embeddings/')
import os
import random
import ujson
import argparse
import pandas as pd
import pickle
import json
import csv
from collections import defaultdict, OrderedDict
import jsonlines

In [73]:
def load_mentions(file): 
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            print(line.keys())
            new_line = {
                'id': line['id'],
                'sentence': line['sentence'],
                'aliases': line['aliases'], 
                'spans': line['spans'],
                'qids': line['qids'],
                #'gold': line['gold'],
                'sent_idx_unq': line['sent_idx_unq'],
                'gold': line['anchor']
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

In [71]:
base_dir = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/'
expt_dir = 'everything/'
filename = 'hybrid_with_bootmentions'

In [72]:
tacred_data_w_bootleg_mentions_base = '{}{}{}.jsonl'.format(base_dir, expt_dir, filename) #TO FILL IN 
base_mentions_df = load_mentions(tacred_data_w_bootleg_mentions_base)


dict_keys(['id', 'sentence', 'aliases', 'spans', 'qids', 'anchor', 'sent_idx_unq', 'aliases_candgen', 'spans_candgen', 'qids_candgen', 'anchors_candgen'])


KeyError: 'gold'

In [53]:
print(base_mentions_df.columns)

Index(['id', 'sentence', 'aliases', 'spans', 'qids', 'gold', 'sent_idx_unq'], dtype='object')


In [54]:
for ind, row in base_mentions_df.iterrows():
    print(row)
    break

id              61b3a5c8c9a882dcfcd2                                                                                                                                                                                                                        
sentence        Tom Thabane resigned in October last year to form the All Basotho Convention ( ABC ) , crossing the floor with 17 members of parliament , causing constitutional monarch King Letsie III to dissolve parliament and call the snap election .
aliases         [tom thabane, all basotho convention, abc, crossing the floor, members of parliament, constitutional monarch, king letsie iii, dissolve parliament, snap election]                                                                          
spans           [[0, 2], [10, 13], [14, 15], [17, 20], [22, 25], [27, 29], [29, 32], [33, 35], [38, 40]]                                                                                                                                         

# add the mentions I want

In [55]:
aliases_to_add = {}
# based on tacred dev
aliases_to_add['millipore'] = {'alias':'merck millipore', 'qid':'Q1669719', 'len':1}
aliases_to_add['US'] = {'alias':'united states', 'qid':'Q30', 'len':1}
aliases_to_add['U.S.'] = {'alias':'united states', 'qid':'Q30', 'len':1}
aliases_to_add['WNO'] = {'alias':"washington national opera", 'qid':'Q386613', 'len':1}

# based on tacred test
# aliases_to_add['UASR'] = {'alias':"university alliance of the silk road", 'qid':'Q28187407', 'len':1}
# aliases_to_add['NTSO'] = {'alias':"national taiwan symphony orchestra", 'qid':'Q6978831', 'len':1}
# aliases_to_add['LIHOP'] = {'alias':"conspiracy theories", 'qid':'Q22763', 'len':1}
# aliases_to_add['ShopperTrak'] = {'alias':"experian footfall", 'qid':'Q5421023', 'len':1}
# aliases_to_add['TPIPL'] = {'alias':"tpi polene", 'qid':'Q7671160', 'len':1}
# aliases_to_add['SouthGobi Energy Resources'] = {'alias':"southgobi resources", 'qid':'Q65041717', 'len':3}
# aliases_to_add['National Energy Administration'] = {'alias':"national energy commission", 'qid':'Q6972446', 'len':3}
# aliases_to_add['Economic Cooperation Organisation'] = {'alias':"economic cooperation organization", 'qid':'Q225950', 'len':3}
# aliases_to_add['Election Complaints Commission'] = {'alias': "election commission", 'qid':'Q935741', 'len':3}


In [56]:
# print(base_mentions_df['sentence'])

In [57]:
pd.set_option('display.max_colwidth', -1)
print("The number of times each alias is added is: ")
for k, v in aliases_to_add.items():
    sub_df = base_mentions_df[base_mentions_df['sentence'].str.contains(k)]
    count = 0
    for ind, row in sub_df.iterrows():
        sentence = row['sentence']
        alias = row['aliases']
        spans = row['spans']
        qids = row['qids']
        gold = row['gold']
        
        k_lst = k.split(' ')
        if v['qid'] not in qids:
            tokens = sentence.split(' ')
            if k_lst[0] in tokens:
                index = tokens.index(k_lst[0])
                if k in ' '.join(tokens[index:index+v['len']]):
                    span = [index, index+v['len']]
                    #print(row)
                    if span not in spans:
                        spans.append(span)
                        alias.append(v['alias'])
                        qids.append(v['qid'])
                        gold.append(True)
                        base_mentions_df.at[ind, 'aliases'] = alias
                        base_mentions_df.at[ind, 'spans'] = spans
                        base_mentions_df.at[ind, 'qids'] = qids
                        base_mentions_df.at[ind, 'gold'] = gold
                        count += 1       
                        
        assert len(alias) == len(gold), print(row)
    print(k, count)


The number of times each alias is added is: 
millipore 0
US 529
U.S. 1328
WNO 0


In [58]:
import json

# path_json = '{}{}TEMPFILE_{}_w_manual_alias.json'.format(base_dir, expt_dir, filename)
# base_mentions_df.to_json(path_json, orient='records')

# with open(path_json) as infile:
#     data = json.load(infile)
#     print(len(data))
    
all_out = '{}{}{}_w_manual_alias.jsonl'.format(base_dir, expt_dir, filename)
with open(all_out, 'w') as outfile:
    for d in data:
        json.save(d, outfile)
        outfile.write('\n')

In [59]:
print(all_out)

/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_092620/manual_alias/all_tacred_w_bootoutput_w_manual_alias.jsonl
